In [1]:
import pandas as pd

df = pd.read_csv("CCR_final.csv")
df

,Timestamp,Elapsed time (minutes),Setpoint (K),Sensor A (K),Sensor B (K),Heater Output % (0-100),Heater Range (integer 0-5),Slope,Intercept,Sensor C (K) or sensor A (sensor unit),Sensor D (K) or sensor B (sensor unit),CCR_ID
0,03/20/2014 03:02:25 PM,0.984532,0.0,282.022,282.108,0.0,5,1.000000,1.000000,0.60091,0.60072,./CCR-05/2014/20140320.txt
1,03/20/2014 03:02:30 PM,1.068290,0.0,282.024,282.110,0.0,5,1.000000,1.000000,0.60091,0.60071,./CCR-05/2014/20140320.txt
2,03/20/2014 03:02:35 PM,1.151215,0.0,282.026,282.112,0.0,5,1.000000,1.000000,0.60090,0.60071,./CCR-05/2014/20140320.txt
3,03/20/2014 03:02:40 PM,1.234957,0.0,282.028,282.115,0.0,5,1.000000,1.000000,0.60090,0.60070,./CCR-05/2014/20140320.txt
4,03/20/2014 03:02:45 PM,1.317882,0.0,282.030,282.117,0.0,5,1.000000,1.000000,0.60089,0.60070,./CCR-05/2014/20140320.txt
...,...,...,...,...,...,...,...,...,...,...,...,...
395695,12/30/2019 10:17:37 AM,4091.354468,300.0,298.495,300.000,12.5,5,-0.000012,0.000076,0.56314,0.55966,./CCR-17/2019/20191227.txt
395696,12/30/2019 10:17:42 AM,4091.437382,300.0,298.496,300.000,12.5,5,-0.000012,0.000071,0.56313,0.55966,./CCR-17/2019/20191227.txt
395697,12/30/2019 10:17:47 AM,4091.521134,300.0,298.497,300.000,12.5,5,-0.000008,0.000040,0.56313,0.55966,./CCR-17/2019/20191227.txt
395698,12/30/2019 10:17:52 AM,4091.604055,300.0,298.498,300.000,12.5,5,-0.000004,0.000008,0.56313,0.55966,./CCR-17/2019/20191227.txt


In [2]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.1 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached grpcio-1.56.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.2 MB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached google_auth-2.21.0-py2.py3-none-any.whl (182 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached Markdown-3.4.3-py3-none-any.whl (93 kB)
  Using cached cachetools-5.3.1-py3-none-any.whl (9.3 kB)
  Using cached pyasn1_modules-0.3.0-py2.py3-none-any.whl (181 kB)
  Using ca

In [4]:
import pandas as pd
import numpy as np
from darts import TimeSeries, metrics
from darts.utils.missing_values import fill_missing_values
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout

from darts.dataprocessing.transformers import Scaler
from darts.models import (
    RNNModel,
    TCNModel,
    TransformerModel,
    NBEATSModel,
    BlockRNNModel,
    VARIMA,  
)

# Load your dataset into a DataFrame (replace 'your_dataset.csv' with your actual file path or name)
# df = pd.read_csv('your_dataset.csv')

# Convert the DataFrame to a Darts TimeSeries object
timestamps = pd.to_datetime(df['Timestamp'])
values = df.drop('Timestamp', axis=1).values
series = TimeSeries.from_dataframe(df, time_col='Timestamp', value_cols=df.columns[1:])

# Fill missing values in the time series using linear interpolation
series = fill_missing_values(series)

# Normalize the time series using Min-Max scaling
scaler = MinMaxScaler()
series = TimeSeries.from_dataframe(series.pd_dataframe().dropna(), time_col='Timestamp', value_cols=df.columns[1:])
series = TimeSeries.from_values(series.timestamp(), scaler.fit_transform(series.univariate_values()).T)

# Split the data into train and test sets
train_data, test_data = train_test_split(series, test_size=0.2, shuffle=False)

class TransferredTransformerModel:
    def __init__(self, input_chunk_length, output_chunk_length, num_heads, num_layers, d_model, d_ff, dropout=0.1):
        self.input_chunk_length = input_chunk_length
        self.output_chunk_length = output_chunk_length
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.d_model = d_model
        self.d_ff = d_ff
        self.dropout = dropout
        self.model = self._build_model()

    def _build_model(self):
        inputs = Input(shape=(self.input_chunk_length, len(series.univariate_values())))
        x = inputs

        # Transformer Encoder
        x = transformer.Encoder(
            num_blocks=self.num_layers,
            d_model=self.d_model,
            num_heads=self.num_heads,
            d_ff=self.d_ff,
            dropout=self.dropout
        )(x)

        # Dense layer for anomaly detection
        x = Dense(self.output_chunk_length, activation='sigmoid')(x)

        model = Model(inputs=inputs, outputs=x)
        model.compile(optimizer='adam', loss='mse')
        return model

    def fit(self, train_data, **kwargs):
        # Convert the time series into supervised training data
        train_data = train_data.pd_dataframe().dropna()
        X = train_data[:-self.output_chunk_length].values.reshape(-1, self.input_chunk_length, len(series.univariate_values()))
        y = train_data[self.input_chunk_length:].values.reshape(-1, self.output_chunk_length, len(series.univariate_values()))

        # Train the model
        self.model.fit(X, y, **kwargs)

    def predict(self, n):
        # Prepare the input data for prediction
        last_chunk = series[-self.input_chunk_length:].univariate_values().reshape(1, self.input_chunk_length, len(series.univariate_values()))

        # Predict the next n chunks of the time series
        predictions = []
        for _ in range(n):
            next_chunk = self.model.predict(last_chunk)[0]

            # Create the next time series chunk with the predicted values
            next_values = np.concatenate([last_chunk[0, -1:, :], next_chunk], axis=0)
            predictions.append(next_values[-self.output_chunk_length:])

            last_chunk = np.concatenate([last_chunk[:, 1:, :], next_values.reshape(1, 1, -1)], axis=1)

        # Create a TimeSeries object for the predictions
        predicted_series = TimeSeries.from_values(timestamps[-n*self.output_chunk_length:], np.concatenate(predictions))

        return predicted_series

# Set hyperparameters for the model
input_chunk_length = 10
output_chunk_length = 1
num_heads = 2
num_layers = 2
d_model = 32
d_ff = 64
dropout = 0.1

# Create an instance of the transferred transformer model
model = TransferredTransformerModel(input_chunk_length, output_chunk_length, num_heads, num_layers, d_model, d_ff, dropout=dropout)

# Train the model
model.fit(train_data, verbose=True)

# Make predictions on the test set
predictions = model.predict(len(test_data))

# Compute the mean squared error (MSE) for the predictions
mse = metrics.mse(predictions, test_data)

print(f"Mean Squared Error (MSE): {mse:.4f}")


/tmp/ipykernel_2181501/3399874454.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  timestamps = pd.to_datetime(df['Timestamp'])


ValueError: could not convert string to float: './CCR-05/2014/20140320.txt'